# Interfacing SkyPortal with ANTARES

**This notebook requires the installation of ANTARES client (https://noao.gitlab.io/antares/client/) and SkyPortal (https://https://skyportal.io). For more detail on programmatic access of SkyPortal with RestAPI, please see (https://skyportal.io/docs/api.html).**



SkyPortal is an astronomical data platform that interactively displays astronomical datasets for annotation, analysis, and discovery. We can ingest ANTARES locus/alert with the SkyPortal API as follows.

The first step is to import ANTARES client, API function, and define API token for SkyPortal

In [1]:
from antares_client.search import get_by_id, get_by_ztf_object_id, search

import requests

#token can be generated by SkyPortal account manager
token = '90de4b61-3629-4271-bea9-b94e23d9559x'

def api(method, endpoint, data=None):
    headers = {'Authorization': f'token {token}'}
    response = requests.request(method, endpoint, json=data, headers=headers)
    return response

The next step is to provide the locus information. The following piece of code will create a candidate in the SkyPortal database using API **POST** function. It will also tell you if the object already exists, and in that case will not attempt to re-create the candidate.

In [ ]:
obj_id="ANT2020pcud2"

locus = get_by_id(obj_id)
print(locus.locus_id,locus.ra,locus.dec)

data = {
"ra": locus.ra,
"dec": locus.dec,
"id": obj_id,
"filter_ids": [ 1 ],
}

response = api('POST', 'http://localhost:5000/api/candidates', data)

print(f'HTTP code: {response.status_code}, {response.reason}')
if response.status_code in (200, 400):
    print(f'JSON response: {response.json()}')


Once the candidate is posted, we can populate the light curve information with all the alerts associated with this locus. This can be done as follows. This piece of code will use the **POST** function to upload photometry from ZTF. Please note that each 

In [ ]:
for alert in locus.alerts:
    if 'ztf_magpsf' in alert.properties:
        if alert.properties['ztf_fid'] == 1:
            filterid='ztfg'
        if alert.properties['ztf_fid'] == 2:
            filterid='ztfr'
        print("AlertID",alert.properties['ztf_candid'])
        phot={
            "magerr": alert.properties['ztf_sigmapsf'],
            "limiting_mag": alert.properties['ztf_diffmaglim'],
            "instrument_id": 6,
            "group_ids": [ 1 ],
            "obj_id": obj_id,
            "mjd": alert.properties['ztf_jd']-2400000.5,
            "magsys": 'ab',
            "filter": filterid,
            "mag": alert.properties['ztf_magpsf']
        }
        response = api('POST', 'http://localhost:5000/api/photometry', phot)
        print(f'HTTP code: {response.status_code}, {response.reason}')
        if response.status_code in (200, 400):
            print(f'JSON response: {response.json()}')


